In [9]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [139]:
class TreeGrid():
    def __init__(self, s) -> None:
        self.grid = defaultdict(
            lambda: {
                "height": int,
                "up": False,
                "down": False,
                "left": False,
                "right": False,
                "scenic": 1,
            }
        )
        for i, row in enumerate(s.splitlines()):
            for j, height in enumerate(row):
                self.grid[(i, j)]["height"] = int(height)
        self.size = list(max(self.grid.keys()))


    def _get_edge_coords(self, from_dir):
        if from_dir.lower() == "up":
            return [(0, j) for j in range(self.size[1]+1)], (1,0)
        elif from_dir.lower() == "down":
            return [(self.size[0], j) for j in range(self.size[1]+1)], (-1,0)
        elif from_dir.lower() == "left":
            return [(i, 0) for i in range(self.size[0]+1)], (0,1)
        elif from_dir.lower() == "right":
            return [(i, self.size[1]) for i in range(self.size[0]+1)], (0,-1)
        else:
            raise Exception(f"{from_dir} could not be recognised. Use 'up', 'down', 'left', 'right'")


    @staticmethod
    def vec_add(vec_a: tuple, vec_b: tuple) -> tuple:
        return tuple(np.array(vec_a) + np.array(vec_b))
        

    def _check_visible_row(self, coord, to_vector, from_dir):
        tallest = -1
        same_height = False
        in_grid = True
        while in_grid:
            if self.grid[coord]["height"] > tallest:
                self.grid[coord][from_dir] = True
                tallest = self.grid[coord]["height"]
            coord = self.vec_add(coord, to_vector)
            in_grid = coord in self.grid.keys()


    def _check_view_distance(self, coord, to_vector):
        block = False
        in_grid = True
        start_height = self.grid[coord]["height"]
        view_distance = 0
        while not block and in_grid:
            new_coord = self.vec_add(coord, to_vector)
            in_grid = new_coord in self.grid.keys()
            if in_grid:
                view_distance += 1
                block = self.grid[new_coord]["height"] >= start_height
            coord = new_coord

        return view_distance


    def check_visible_from_edge(self, from_dir):
        initial_coords, to_vector = self._get_edge_coords(from_dir)
        for coord in initial_coords:
            self._check_visible_row(coord, to_vector, from_dir)


    def check_visible_from_point(self, coord):
        for from_dir in ["up", "down", "left", "right"]:
            _, to_vector = self._get_edge_coords(from_dir)
            self.grid[coord]["scenic"] *= self._check_view_distance(coord, to_vector)

In [140]:
with open("test_input.txt", "r") as f:
    s = f.read()

grid = TreeGrid(s)
from_dirs = ["up", "down", "left", "right"]
for from_dir in from_dirs:
    grid.check_visible_from_edge(from_dir)
    
visible_coords = [coord for coord, d in grid.grid.items() if any([d[from_dir] for from_dir in from_dirs])]
assert len(visible_coords) == 21, len(visible_coords)

for coord in grid.grid.keys():
    grid.check_visible_from_point(coord)
max_scenic = max([d["scenic"] for coord, d in grid.grid.items()])
assert max_scenic == 8, max_scenic

In [141]:
with open("input.txt", "r") as f:
    s = f.read()

grid = TreeGrid(s)
from_dirs = ["up", "down", "left", "right"]
for from_dir in from_dirs:
    grid.check_visible_from_edge(from_dir)
    
visible_coords = [coord for coord, d in grid.grid.items() if any([d[from_dir] for from_dir in from_dirs])]
print(len(visible_coords))

for coord in grid.grid.keys():
    grid.check_visible_from_point(coord)
max_scenic = max([d["scenic"] for coord, d in grid.grid.items()])
print(max_scenic)

1859
332640
